In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Prosta analiza eksploracyjna (EDA) Twoich źródeł:
- rozmiar zbioru
- lista kolumn
- brakujące wartości
- duplikaty po URL
- rozkład artykułów w czasie (miesiąc / rok-miesiąc)
"""

import pandas as pd
from pathlib import Path

# ================== KONFIGURACJA ŚCIEŻEK ==================

FILES = {
    "obserwatorfinansowy": "wyniki LLM/obserwatorfinansowy_llm_extracted_clean.csv",
    "businessinsider":      "wyniki LLM/businessinsider_llm_extracted.csv",
    "pap":                  "wyniki LLM/pap_llm_extracted.csv",
    "tvn24":                "wyniki LLM/tvn24_llm_extracted.csv",
    "money_pl":             "wyniki LLM/money_llm_extracted.csv",
    "bankier":              "wyniki LLM/bankier_llm_extracted.csv",
}

# Jeśli u siebie masz inne ścieżki, po prostu zmień wartości w słowniku FILES.


# ================== FUNKCJE POMOCNICZE ==================

def find_date_column(df: pd.DataFrame):
    """
    Próbuje znaleźć kolumnę z datą na podstawie nazwy,
    a potem sprawdza, czy da się ją sparsować do datetime.
    Zwraca: (nazwa_kolumny, seria_datetime) albo (None, None)
    """
    if df.empty:
        return None, None

    # Kandydaci po nazwach kolumn
    candidates = [
        c for c in df.columns
        if any(
            key in c.lower()
            for key in ["date", "data", "time", "czas", "published", "pubdate"]
        )
    ]

    for col in candidates:
        try:
            parsed = pd.to_datetime(
                df[col],
                errors="coerce",
                utc=True  # Changed to True to handle mixed timezones
            )
            # Akceptujemy, jeśli co najmniej połowa wierszy da się sparsować
            if parsed.notna().mean() > 0.5:
                return col, parsed
        except Exception:
            continue

    return None, None


def find_url_column(df: pd.DataFrame):
    """
    Szuka kolumny z URL na podstawie nazwy.
    Zwraca nazwę kolumny lub None.
    """
    candidates = [
        c for c in df.columns
        if any(key in c.lower() for key in ["url", "link", "href"])
    ]
    return candidates[0] if candidates else None


def basic_info(df: pd.DataFrame):
    """
    Zwraca słownik z podstawowymi informacjami o dataframe.
    """
    info = {}
    info["rows"], info["cols"] = df.shape
    info["columns"] = list(df.columns)

    # Braki (tylko procentowo, żeby nie zaspamować)
    na_ratio = df.isna().mean().sort_values(ascending=False)
    info["na_ratio"] = na_ratio

    return info


def monthly_stats(dates: pd.Series):
    """
    Przyjmuje serię datetime i zwraca liczbę artykułów per rok-miesiąc.
    """
    # Rok-miesiąc jako string YYYY-MM
    month = dates.dt.to_period("M").astype(str)
    return month.value_counts().sort_index()


def print_section(title: str, char: str = "="):
    print("\n" + char * 80)
    print(title)
    print(char * 80 + "\n")


# ================== GŁÓWNA FUNKCJA ANALIZY ==================

def analyze_file(label: str, path: str):
    print_section(f"[{label}] Analiza pliku: {path}", "=")

    p = Path(path)
    if not p.exists():
        print(f"[WARN] Plik {p} nie istnieje. Pomiń lub popraw ścieżkę.\n")
        return

    # Wczytanie danych
    try:
        df = pd.read_csv(p)
    except UnicodeDecodeError:
        # awaryjnie inny encoding
        df = pd.read_csv(p, encoding="utf-8", errors="ignore")

    info = basic_info(df)

    # ---- Podstawowe info ----
    print(f"Liczba wierszy: {info['rows']}")
    print(f"Liczba kolumn: {info['cols']}\n")

    print("Kolumny:")
    for col in info["columns"]:
        print(f"  - {col}")
    print()

    # ---- Braki danych ----
    print("Braki danych (procentowo, TOP 10 kolumn z największym udziałem NaN):")
    na_ratio = info["na_ratio"].head(10).round(3)
    print(na_ratio.to_string())
    print()

    # ---- URL / duplikaty ----
    url_col = find_url_column(df)
    if url_col is not None:
        total = len(df)
        unique_urls = df[url_col].nunique(dropna=True)
        dup_count = total - unique_urls
        print(f"Kolumna URL: {url_col}")
        print(f"  unikalnych URL-i: {unique_urls}")
        print(f"  duplikatów po URL: {dup_count}\n")
    else:
        print("Nie znaleziono oczywistej kolumny z URL (url/link/href).\n")

    # ---- Data / rozkład miesięczny ----
    date_col, parsed_dates = find_date_column(df)
    if date_col is not None:
        print(f"Znaleziono kolumnę z datą: {date_col}")
        print(f"  odsetek poprawnie sparsowanych dat: {parsed_dates.notna().mean():.3f}")

        valid = parsed_dates.dropna()
        if not valid.empty:
            print(f"  zakres dat: {valid.min().date()}  ->  {valid.max().date()}\n")

            monthly = monthly_stats(valid)
            print("Liczba artykułów per miesiąc (YYYY-MM):")
            print(monthly.to_string())
            print()
        else:
            print("  Wszystkie daty są NaT po parsowaniu – trzeba sprawdzić format ręcznie.\n")
    else:
        print("Nie udało się automatycznie znaleźć kolumny z datą.\n")

    # ---- Przykładowe rozkłady innych kolumn (jeśli są) ----
    for candidate in ["source", "economist", "author", "site", "indicator", "indicator_type"]:
        if candidate in df.columns:
            print(f"Rozkład wartości w kolumnie '{candidate}' (TOP 15):")
            print(df[candidate].value_counts(dropna=False).head(15).to_string())
            print()

    print("\n" + "-" * 80 + "\n")


# ================== URUCHOMIENIE ANALIZY DLA WSZYSTKICH PLIKÓW ==================

if __name__ == "__main__":
    for label, path in FILES.items():
        analyze_file(label, path)



[obserwatorfinansowy] Analiza pliku: wyniki LLM/obserwatorfinansowy_llm_extracted_clean.csv

Liczba wierszy: 514
Liczba kolumn: 10

Kolumny:
  - url
  - query
  - title_search
  - snippet_search
  - google_detected_date
  - has_forecast
  - main_topic
  - country
  - forecasts_json
  - processed_idx

Braki danych (procentowo, TOP 10 kolumn z największym udziałem NaN):
google_detected_date    1.000
processed_idx           0.981
url                     0.000
query                   0.000
title_search            0.000
snippet_search          0.000
has_forecast            0.000
main_topic              0.000
country                 0.000
forecasts_json          0.000

Kolumna URL: url
  unikalnych URL-i: 514
  duplikatów po URL: 0

Nie udało się automatycznie znaleźć kolumny z datą.


--------------------------------------------------------------------------------


[businessinsider] Analiza pliku: wyniki LLM/businessinsider_llm_extracted.csv

Liczba wierszy: 719
Liczba kolumn: 9

Kolumny:

/var/folders/q7/p3s6dfbn0dv04bh3j4kgrz2c0000gn/T/ipykernel_1502/2409771170.py:98: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  month = dates.dt.to_period("M").astype(str)
/var/folders/q7/p3s6dfbn0dv04bh3j4kgrz2c0000gn/T/ipykernel_1502/2409771170.py:98: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  month = dates.dt.to_period("M").astype(str)
/var/folders/q7/p3s6dfbn0dv04bh3j4kgrz2c0000gn/T/ipykernel_1502/2409771170.py:98: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  month = dates.dt.to_period("M").astype(str)
